In [1]:
import requests
import os
import gradio as gr

c:\Users\Lenovo\AppData\Local\Programs\Python\Python310\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [13]:
def scanFileAtVT(file_path):
    ##sending the file for scanning
    url = 'https://www.virustotal.com/vtapi/v2/file/scan'
    params = {'apikey': '453af95ed16995efc1d0894c5df35a4e9374de3792225c4a48f46c1f484afbb7'}
    files = {'file': (file_path, open(file_path, 'rb'))}
    response = requests.post(url, files=files, params=params)
    response_json = response.json()
    if response.status_code != 200:
        print("Error submitting file for scanning")
    
    ##sending the response to get the report
    sha256 = response_json['sha256']
    url = "https://www.virustotal.com/vtapi/v2/file/report"
    params = {'apikey': '453af95ed16995efc1d0894c5df35a4e9374de3792225c4a48f46c1f484afbb7', 'resource':sha256}
    response1 = requests.get(url, params=params)
    data = response1.json()

    ##printing how many sources found it malicious
    try:
        countpos = 0
        counttotal = 0
        for i in data['scans']:
            counttotal += 1
            if(data['scans'][i]['detected'] == True):
                countpos += 1
        print( f'{countpos} sources out of {counttotal} found ' + os.path.basename(file_path) + ' malicious') 
    except KeyError:
        print("The file couldn't be scanned")

In [3]:
def getFiles(path):
    for filename in os.listdir(path):
        print("--" + filename)
        fullname = os.path.join(path, filename)
        if os.path.isdir(fullname): 
            getFiles(fullname)
        else:
            scanFileAtVT(fullname) 

In [12]:
def dir_or_file(path):
    if os.path.isfile(path):
        scanFileAtVT(path)
    if os.path.isdir(path):
        getFiles(path)
dir_or_file(r"C:\Users\Lenovo\Documents\Diabetes prediction with KNN.docx")
demo = gr.Interface(
    fn = dir_or_file,
    inputs = "text",
    outputs = "text",
)

demo.launch()

Running on local URL:  http://127.0.0.1:7867

To create a public link, set `share=True` in `launch()`.
